In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")

def check_text_grammar(text):
    doc = nlp(text)
    issues = []
    
    for sent in doc.sents:
        sentence = sent.text.strip()
        errors = []
        

        if "hope" in sentence.lower() and not any(
            t.text.lower() == "i" and t.head.text.lower() == "hope" 
            for t in sent
        ):
            errors.append("Use 'I hope' instead of 'Hope' at sentence start")
        

        for token in sent:
            if token.dep_ == "nsubj" and token.text.lower() == "you":
                if not any(t.text.lower() in ["will", "would"] for t in token.head.rights):
                    errors.append("Add modal verb (e.g., 'I hope you will enjoy')")
        

        awkward_phrases = {
            "as my deepest wishes": "as I sincerely hope",
            "thank your message": "thank you for your message",
            "i am very appreciated": "I greatly appreciate"
        }
        for phrase, correction in awkward_phrases.items():
            if phrase in sentence.lower():
                errors.append(f"Replace '{phrase}' with '{correction}'")

        for token in sent:
            if token.text == "appreciated" and "am" in [t.text.lower() for t in token.lefts]:
                errors.append("Use active voice: 'I appreciate' not 'I am appreciated'")
        
        if errors:
            issues.append({
                "sentence": sentence,
                "errors": errors
            })
    
    return issues

def print_grammar_issues(text):
    issues = check_text_grammar(text)
    
    if not issues:
        print("✓ No grammar issues found")
        return
    
    print(f"Found {len(issues)} problematic sentences:\n")
    for i, issue in enumerate(issues, 1):
        print(f"{i}. Sentence: '{issue['sentence']}'")
        for error in issue["errors"]:
            print(f"   - {error}")
        print("\n" + "-"*50 + "\n")
    
    print("\nSuggested corrected text:")
    print(generate_corrections(text, issues))

def generate_corrections(text, issues):
    """Generate corrected text based on issues found"""
    corrected = text
    for issue in issues:
        original = issue["sentence"]
        fixed = original
        
        if "Use 'I hope'" in "\n".join(issue["errors"]):
            fixed = fixed.replace("Hope", "I hope").replace("hope", "I hope", 1)
        
        if "Add modal verb" in "\n".join(issue["errors"]):
            fixed = fixed.replace("you,", "you will,").replace("you ", "you will ")
        
        for phrase in ["as my deepest wishes", "thank your message", "i am very appreciated"]:
            if phrase in original.lower():
                replacements = {
                    "as my deepest wishes": "as I sincerely hope",
                    "thank your message": "thank you for your message",
                    "i am very appreciated": "I greatly appreciate"
                }
                fixed = fixed.replace(phrase, replacements[phrase])
        
        corrected = corrected.replace(original, fixed)
    
    return corrected
text = """Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in 
our lives. Hope you too, to enjoy it as my deepest wishes. 
Thank your message to show our words to the doctor, as his next contract checking, to all of us. 
I got this message to see the approved message. In fact, I have received the message from the 
professor, to show me, this, a couple of days ago.  I am very appreciated  the full support of the 
professor, for our Springer proceedings publication"""

print_grammar_issues(text)




Found 3 problematic sentences:

1. Sentence: 'Hope you too, to enjoy it as my deepest wishes.'
   - Use 'I hope' instead of 'Hope' at sentence start
   - Replace 'as my deepest wishes' with 'as I sincerely hope'

--------------------------------------------------

2. Sentence: 'Thank your message to show our words to the doctor, as his next contract checking, to all of us.'
   - Replace 'thank your message' with 'thank you for your message'

--------------------------------------------------

3. Sentence: 'I am very appreciated  the full support of the 
professor, for our Springer proceedings publication'
   - Replace 'i am very appreciated' with 'I greatly appreciate'

--------------------------------------------------


Suggested corrected text:
Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in 
our lives. I I hope you too, to enjoy it as I sincerely hope. 
Thank your message to show our words to the doctor, as his next contract che